In [1]:
import json
import time

from kafka import KafkaProducer

In [2]:
def json_serializer(data):
    return json.dumps(data).encode("utf-8")

In [3]:
server = "localhost:9092"

producer = KafkaProducer(bootstrap_servers=[server], value_serializer=json_serializer)

In [4]:
producer.bootstrap_connected()

True

In [5]:
t0 = time.time()
topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)
t1 = time.time()
print(f'sending messages took {(t1 - t0):.2f} seconds')

t2 = time.time()
producer.flush()
t3 = time.time()
print(f'flushing took {(t3 - t2):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
sending messages took 0.55 seconds
flushing took 0.00 seconds


In [5]:
import pandas as pd

In [6]:
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', compression='gzip')

/var/folders/0s/9k5wnkxx22l_g6lmt5xw673m0000gq/T/ipykernel_97733/2373606625.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', compression='gzip')


In [7]:
df_green = df_green[['lpep_pickup_datetime',
                     'lpep_dropoff_datetime',
                     'PULocationID',
                     'DOLocationID',
                     'passenger_count',
                     'trip_distance',
                     'tip_amount']]

In [8]:
import datetime

In [ ]:
topic_name = 'green-trips'
t0 = time.time()
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    row_dict['timetstamp'] = datetime.datetime.now().timestamp()
    producer.send(topic_name, value=row_dict)
t1 = time.time()

print(f'Sending these messages took {(t1 - t0):.2f} seconds')

-------------------------------------------
Batch: 1
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



...

-------------------------------------------
Batch: 7
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 8
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 9
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+

Sending these messages took 26.56 seconds


-------------------------------------------
Batch: 10
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 11
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



In [10]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/jessicadesilva/.ivy2/cache
The jars for the packages stored in: /Users/jessicadesilva/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-98f5a531-3d10-401f-b029-cb7ff87ba24c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 224ms :: art

In [33]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [34]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType()) \
    .add("timestamp", types.TimestampType())

In [35]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [15]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

In [16]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/03/12 18:30:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0s/9k5wnkxx22l_g6lmt5xw673m0000gq/T/temporary-baeac6e5-d6b0-42ae-997b-642160d1bcc0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/12 18:30:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/03/12 18:30:50 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0, timestamp=None)


In [17]:
query.stop()

In [ ]:
query = green_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

24/03/12 18:53:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0s/9k5wnkxx22l_g6lmt5xw673m0000gq/T/temporary-c61d9cd7-5933-419a-8a58-d92d71b1bc68. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/12 18:53:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;
Project [data#517.lpep_pickup_datetime AS lpep_pickup_datetime#519, data#517.lpep_dropoff_datetime AS lpep_dropoff_datetime#520, data#517.PULocationID AS PULocationID#521, data#517.DOLocationID AS DOLocationID#522, data#517.passenger_count AS passenger_count#523, data#517.trip_distance AS trip_distance#524, data#517.tip_amount AS tip_amount#525, data#517.timestamp AS timestamp#526]
+- Project [from_json(StructField(lpep_pickup_datetime,StringType,true), StructField(lpep_dropoff_datetime,StringType,true), StructField(PULocationID,IntegerType,true), StructField(DOLocationID,IntegerType,true), StructField(passenger_count,DoubleType,true), StructField(trip_distance,DoubleType,true), StructField(tip_amount,DoubleType,true), StructField(timestamp,TimestampType,true), cast(value#504 as string), Some(America/Los_Angeles)) AS data#517]
   +- StreamingRelationV2 org.ap

-------------------------------------------
Batch: 1
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



...

-------------------------------------------
Batch: 61
-------------------------------------------
+--------------------+------------+-----+
|              window|DOLocationID|count|
+--------------------+------------+-----+
|{2024-03-12 20:00...|         194|  244|
|{2024-03-12 20:05...|          32|  608|
|{2024-03-12 20:05...|          63|  657|
|{2024-03-12 20:05...|          42|15942|
|{2024-03-12 20:05...|         177| 1520|
|{2024-03-12 20:00...|          11|  412|
|{2024-03-12 20:00...|         160| 1699|
|{2024-03-12 20:05...|          19|  352|
|{2024-03-12 20:05...|          96|  203|
|{2024-03-12 20:00...|          62| 1680|
|{2024-03-12 20:00...|          23|   15|
|{2024-03-12 20:05...|         162| 1822|
|{2024-03-12 20:00...|         182|  955|
|{2024-03-12 20:00...|          30|   18|
|{2024-03-12 20:05...|         166|10845|
|{2024-03-12 20:05...|         101|  254|
|{2024-03-12 20:00...|          28| 2292|
|{2024-03-12 20:05...|         140| 2853|
|{2024-03-12 20:05..

-------------------------------------------
Batch: 46
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|194         |244  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|32          |608  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|63          |657  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|42          |15942|
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|177         |1520 |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|11          |412  |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|160         |1699 |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|19          |352  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|96          |203  |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|62          |1680 |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|23          |15   |
|{2024

-------------------------------------------
Batch: 48
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|194         |244  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|32          |608  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|63          |657  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|42          |15942|
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|177         |1520 |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|11          |412  |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|160         |1699 |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|19          |352  |
|{2024-03-12 20:05:00, 2024-03-12 20:10:00}|96          |203  |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|62          |1680 |
|{2024-03-12 20:00:00, 2024-03-12 20:05:00}|23          |15   |
|{2024

-------------------------------------------
Batch: 50
-------------------------------------------
+--------------------+------------+-----+
|              window|DOLocationID|count|
+--------------------+------------+-----+
|{2024-03-12 20:00...|         194|  244|
|{2024-03-12 20:05...|          32|  608|
|{2024-03-12 20:05...|          63|  657|
|{2024-03-12 20:05...|          42|15942|
|{2024-03-12 20:05...|         177| 1520|
|{2024-03-12 20:00...|          11|  412|
|{2024-03-12 20:00...|         160| 1699|
|{2024-03-12 20:05...|          19|  352|
|{2024-03-12 20:05...|          96|  203|
|{2024-03-12 20:00...|          62| 1680|
|{2024-03-12 20:00...|          23|   15|
|{2024-03-12 20:05...|         162| 1822|
|{2024-03-12 20:00...|         182|  955|
|{2024-03-12 20:00...|          30|   18|
|{2024-03-12 20:05...|         166|10845|
|{2024-03-12 20:05...|         101|  254|
|{2024-03-12 20:00...|          28| 2292|
|{2024-03-12 20:05...|         140| 2853|
|{2024-03-12 20:05..

ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv4 ('127.0.0.1', 9092)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "/Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/.venv/lib/python3.12/site-packages/kafka/conn.py", line 1090, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 54] Connection reset by peer
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Error receiving network data closing socket
Traceback (most recent call last):
  File "/Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/.venv/lib/python3.12/site-packages/kafka/conn.py", line 1090, in _recv
    data = self._sock.recv(self.config['sock_chunk_bytes'])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 54] Connection 

In [36]:
popular_destinations = green_stream.groupBy(
        F.window(
            timeColumn=green_stream.timestamp,
            windowDuration='5 minutes'
        ),
        green_stream.DOLocationID,
    ).count()

In [37]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/03/12 18:52:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0s/9k5wnkxx22l_g6lmt5xw673m0000gq/T/temporary-376019af-a1cc-4b7f-a555-cb4c4453de74. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/12 18:52:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/12 18:52:47 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/.venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/.venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 